In [19]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
import pandas as pd
import os

spark = SparkSession.builder \
    .appName("NetFlowReader") \
    .enableHiveSupport() \
    .getOrCreate()

es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.nodes","elksj-elasticsearch:9200")
)

In [20]:
# flowDF = es_reader.load("netflow-*/")
flowDF = es_reader.load("zeek-*/")
# flowDF.describe( ).show()
testflowDF = flowDF.limit(3) #限制log数量，生成一个小规模的DataFrame
testflowDF.show()
# flowDF.select( "ssl" ).show( false)

In [36]:
# Convert Spark Dataframe to Pandas Dataframe
# flowpdDF = flowDF.toPandas()
flowpdDF = testflowDF.toPandas()
print(flowpdDF.loc[0])

In [65]:
# Load enrichment data as a Pandas DF
asnDF = pd.read_csv('asn_enrich.csv', delimiter='|').set_index('ASN')
asn_dict = asnDF.to_dict()

In [41]:
# Enrich Data - anything unknown is assigned to -Reserved AS- with Country Code ZZ
# flowpdDF['SRC_ORG'] = flowpdDF['SRC_AS'].apply(lambda x: asn_dict['ORG'][x] if x in asn_dict['ORG'] else '-Reserved AS-')
# flowpdDF['SRC_COUNTRY'] = flowpdDF['SRC_AS'].apply(lambda x: asn_dict['COUNTRY'][x] if x in asn_dict['ORG'] else 'ZZ')

# flowpdDF['DST_ORG'] = flowpdDF['DST_AS'].apply(lambda x: asn_dict['ORG'][x] if x in asn_dict['ORG'] else '-Reserved AS-')
# flowpdDF['DST_COUNTRY'] = flowpdDF['DST_AS'].apply(lambda x: asn_dict['COUNTRY'][x] if x in asn_dict['ORG'] else 'ZZ')

In [42]:
flowpdDF.head()

,34,42,@timestamp,@version,BGP_IPV4_NEXT_HOP,DST_AS,EXPORTER_IPV4_ADDRESS,FIRST_SWITCHED,FLOW_END_MILLISECONDS,FLOW_START_MILLISECONDS,...,OUT_PKTS,PROTOCOL,SRC_AS,SRC_TOS,TCP_FLAGS,log_type,SRC_ORG,SRC_COUNTRY,DST_ORG,DST_COUNTRY
0,1,9252,2020-11-04 23:24:26.835,1,0.0.0.0,0,172.18.0.9,1604532098,1604532098264,1604532098258,...,0,6,0,0,27,netflow,-Reserved AS-,ZZ,-Reserved AS-,ZZ
1,1,7694,2020-11-04 23:24:26.180,1,0.0.0.0,0,172.18.0.9,1604532097,1604532114532,1604532097078,...,6,6,8075,32,24,netflow,MICROSOFT-CORP-MSN-AS-BLOCK,US,-Reserved AS-,ZZ
2,1,9254,2020-11-04 23:24:26.835,1,0.0.0.0,0,172.18.0.9,1604532098,1604532098264,1604532098260,...,0,6,0,0,27,netflow,-Reserved AS-,ZZ,-Reserved AS-,ZZ
3,1,7697,2020-11-04 23:24:26.180,1,0.0.0.0,0,172.18.0.9,1604532097,1604532097079,1604532097079,...,1,17,0,0,0,netflow,-Reserved AS-,ZZ,-Reserved AS-,ZZ
4,1,9257,2020-11-04 23:24:26.835,1,0.0.0.0,15169,172.18.0.9,1604532098,1604532104085,1604532098263,...,3131,6,0,0,27,netflow,-Reserved AS-,ZZ,GOOGLE,US


In [56]:
# Convert the pandas DF back to Spark
# flowDF = spark.createDataFrame(flowpdDF)
#构建 schema,使用显式方法规定要转换成的Spark DF形式，避免特殊数据的影响
schema = StructType([
    StructField("column_1", StringType(), True),
    StructField("column_2", StringType(), True),
    StructField("column_3", StringType(), True),
    StructField("column_4", StringType(), True),
    StructField("column_5", StringType(), True),
    StructField("column_6", StringType(), True),
    StructField("column_7", StringType(), True),
    StructField("column_8", StringType(), True),
    StructField("column_9", StringType(), True),
    StructField("column_10", StringType(), True),
    StructField("column_11", StringType(), True),
    StructField("column_12", StringType(), True),
    StructField("column_13", StringType(), True),
    StructField("column_14", StringType(), True),
    StructField("column_15", StringType(), True),
    StructField("column_16", StringType(), True),
    StructField("column_17", StringType(), True),
    StructField("column_18", StringType(), True),
    StructField("column_19", StringType(), True),
    StructField("column_20", StringType(), True),
    StructField("column_21", StringType(), True)
])
flowDF = spark.createDataFrame(flowpdDF, schema=schema)

In [66]:
# # Store to elasticsearch
# (flowDF.write.format('org.elasticsearch.spark.sql')
#              .option("es.nodes","elksj-elasticsearch:9200")
#              .option('es.resource', '%s/%s' % ('enriched_netflow', 'netflow'))
#              .save())